In [1]:
from collections import Counter

import pandas as pd
import torch
from kge.job.eval import EvaluationJob
from kge.model import KgeModel
from kge.util.io import load_checkpoint
from functools import reduce

PATH = "/work/dhuynh/Workspaces/Thesis/kge/local/experiments/error_analysis"

checkpoint = load_checkpoint(f'{PATH}/best_models/ICLR20/ComplEx/FB237/ER_standard/checkpoint_best.pt')
model = KgeModel.create_from(checkpoint)

# load data
model.dataset.relation_strings(torch.Tensor([1,]).long())
model.dataset.entity_strings(torch.Tensor([1,]).long())

meta_relation = model.dataset._meta['relation_strings']
meta_entity = model.dataset._meta['entity_strings']


Loading configuration of dataset fb15k-237 from /work/dhuynh/Workspaces/Thesis/kge/data/fb15k-237 ...
Loaded 237 keys from map relation_ids
Setting reciprocal_relations_model.base_model.entity_embedder.dropout to 0., was set to -0.18433461524546146.
Loaded 237 keys from map relation_strings
Loaded 14541 keys from map entity_ids
Loaded 14541 keys from map entity_strings


In [2]:

subject_name = 'Stan Lee'
relation_name = '/business/business_operation/operating_income./measurement_unit/dated_money_value/currency'
object_name = 'X-Men (Action/Adventure Film)'

id_subject = meta_entity.index(subject_name)
id_relation = meta_relation.index(relation_name)
id_object = meta_entity.index(object_name)

s = torch.Tensor([id_subject]).long()
p = torch.Tensor([id_relation]).long()
o = torch.Tensor([id_object]).long()

print(model.dataset.entity_strings(s))
print(model.dataset.relation_strings(p))
print(model.dataset.entity_strings(o))

['Stan Lee']
['/business/business_operation/operating_income./measurement_unit/dated_money_value/currency']
['X-Men (Action/Adventure Film)']


In [3]:
true_score = model.score_spo(s, p, o, "o")
true_score

tensor([1.6331], grad_fn=<ViewBackward0>)

In [4]:
scores = model.score_sp(s, p)                
predicted_o = torch.topk(scores, 15)
predicted_o_indices = predicted_o.indices
predicted_o_score =  predicted_o.values.detach().cpu().numpy().tolist()
list(model.dataset.entity_strings(predicted_o_indices))


[array(['The Lord of the Rings: The Two Towers', 'Austria-Hungary',
        'France', 'Comedy', 'Scotland', 'United Kingdom',
        'European Union Member States', 'Ottoman Empire', 'Italy',
        'Holy Roman Empire', 'Northern Ireland', 'United States Dollar',
        'The Mummy', 'Video', '38th Daytime Emmy Awards'], dtype='<U37')]

In [17]:
import copy
eval_type = 'train'
model.config.set("eval.split", eval_type)
valid_data = model.dataset.split(eval_type)
new_dataset = copy.deepcopy(model.dataset)
new_dataset._triples[eval_type] = valid_data[[1, 17, 18, 19, 20, 136, 20451, 12366]]
# new_dataset._triples[eval_type] = valid_data[valid_data[:, 1] == id_relation]
# new_dataset._triples[eval_type] = valid_data[[1]]
# print(model.dataset.entity_strings(new_dataset._triples[eval_type][:, 0]))
# print(model.dataset.relation_strings(new_dataset._triples[eval_type][:, 1]))
# print(model.dataset.entity_strings(new_dataset._triples[eval_type][:, 2]))
# new_dataset._triples[eval_type]

In [18]:
so_index = model.dataset.index(f"{eval_type}_so_to_p")
so_index.get_all(new_dataset._triples[eval_type][:, [0, 2]])

tensor([[  0,   1],
        [  1,   6],
        [  2,  16],
        [  3,   8],
        [  4,   7],
        [  5,  24],
        [  5,  66],
        [  6,  24],
        [  6,  66],
        [  6, 160],
        [  7,  45],
        [  7,  53],
        [  7,  77]], dtype=torch.int32)

In [19]:
new_dataset._triples[eval_type]

tensor([[   2,    1,    3],
        [  34,    6,   35],
        [  36,   16,   32],
        [  37,    8,   38],
        [  39,    7,   40],
        [ 248,   66,  249],
        [1798,   66, 1327],
        [1798,   45, 4657]], dtype=torch.int32)

In [27]:
score = model.score_so(new_dataset._triples[eval_type][:, 0], new_dataset._triples[eval_type][:, 2])
torch.topk(score, 31).indices

tensor([[293,  81,   1, 105, 318, 183, 339, 456, 146,  80, 201, 184, 376, 210,
         186, 275,  84, 424,  56,  12, 399, 116, 109, 223, 410, 243, 232, 136,
         382, 465, 205],
        [ 91, 120, 293, 225, 373, 390, 166,   6, 392,  34, 108,  38, 318, 460,
         420, 102, 167, 346, 321,  79, 423, 387, 238, 353, 249, 181, 421, 127,
         173, 436,  80],
        [324, 205,  16, 135, 315, 133,  33, 219, 234, 195, 111, 127, 206, 118,
          62, 143, 427, 139, 252, 146, 150,  91, 398, 120, 199, 291, 182, 185,
          81, 194,  28],
        [ 56, 383, 293, 318,  81, 465, 181, 228, 182, 245,   8, 230, 235, 412,
         210, 467, 290, 425, 196, 418, 188, 429, 175, 146, 240, 471,  53,   3,
         455, 472, 152],
        [212,  81, 210, 167, 181,  56,   7, 267, 318, 178, 190, 127, 235, 462,
         310, 459, 297, 292, 434, 431,  61, 217, 111,  62, 189, 418, 144, 154,
         389,  69, 218],
        [ 24, 167,  33,  39, 120, 219, 195, 110, 139, 324, 205, 225, 206, 166,
      

In [23]:
eval.all_ranks

{'sp_to_o': tensor([ 0,  3,  0, 12, 28,  0,  1,  0]),
 'po_to_s': tensor([ 0,  6, 90,  8,  2, 68, 87,  0]),
 'so_to_p': tensor([ 2,  7,  2, 10,  6, 30, 16, 13])}

In [24]:
eval.all_ranks_filt

{'sp_to_o': tensor([0, 0, 0, 0, 0, 0, 0, 0]),
 'po_to_s': tensor([0, 0, 0, 0, 0, 0, 0, 0]),
 'so_to_p': tensor([ 2,  7,  2, 10,  6, 29, 14, 12])}

In [25]:
eval.all_ranks_filt_test

{'sp_to_o': tensor([0, 0, 0, 0, 0, 0, 0, 0]),
 'po_to_s': tensor([0, 0, 0, 0, 0, 0, 0, 0]),
 'so_to_p': tensor([ 2,  7,  2, 10,  6, 29, 14, 12])}

In [22]:
from kge.job.eval import EvaluationJob

eval = EvaluationJob.create(config=model.config, 
                                    dataset=new_dataset, 
                                    model=model)
eval._prepare()
eval._run()

[172246f5]   93372 distinct sp pairs in train
[172246f5]   56317 distinct po pairs in train
[172246f5]   12079 distinct sp pairs in valid
[172246f5]   8033 distinct po pairs in valid
[172246f5]   13737 distinct sp pairs in test
[172246f5]   9113 distinct po pairs in test
[172246f5]   20354 distinct so pairs in test
[36c46680] Evaluating on train data (epoch -1)...
[172246f5] Loaded relation index
[172246f5]   108 relations of type M-N
[172246f5]   26 relations of type 1-N
[172246f5]   86 relations of type M-1
[172246f5]   17 relations of type 1-1
[36c46680]   batches: 1, mrr (filt.): 0.361 (0.717), hits@1: 0.250 (0.667), hits@1000: 1.000 (1.000)
[36c46680]   entry_id: 92c67175-affa-48bf-ae62-e25b5d3d7a46
[36c46680]   epoch: -1
[36c46680]   epoch_time: 0.06534528732299805
[36c46680]   event: eval_completed
[36c46680]   filter_splits:
[36c46680]   - train
[36c46680]   - valid
[36c46680]   hits_at_1: 0.25
[36c46680]   hits_at_10: 0.625
[36c46680]   hits_at_100: 1.0
[36c46680]   hits_at_10

/work/dhuynh/Workspaces/Thesis/kge/kge/job/eval_entity_ranking.py:743: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').



{'job_id': '36c46680-fb22-4a0f-af06-55d872254afd',
 'job': 'train',
 'type': 'entity_ranking',
 'scope': 'epoch',
 'split': 'train',
 'filter_splits': ['train', 'valid'],
 'epoch': -1,
 'batches': 1,
 'size': 8,
 'epoch_time': 0.06534528732299805,
 'event': 'eval_completed',
 'mean_rank': 17.291666666666668,
 'mean_reciprocal_rank': 0.3613956769307454,
 'hits_at_1': 0.25,
 'hits_at_3': 0.4166666666666667,
 'hits_at_10': 0.625,
 'hits_at_50': 0.875,
 'hits_at_100': 1.0,
 'hits_at_200': 1.0,
 'hits_at_300': 1.0,
 'hits_at_400': 1.0,
 'hits_at_500': 1.0,
 'hits_at_1000': 1.0,
 'mean_rank_filtered': 4.416666666666667,
 'mean_reciprocal_rank_filtered': 0.7167648474375407,
 'hits_at_1_filtered': 0.6666666666666666,
 'hits_at_3_filtered': 0.75,
 'hits_at_10_filtered': 0.8333333333333334,
 'hits_at_50_filtered': 1.0,
 'hits_at_100_filtered': 1.0,
 'hits_at_200_filtered': 1.0,
 'hits_at_300_filtered': 1.0,
 'hits_at_400_filtered': 1.0,
 'hits_at_500_filtered': 1.0,
 'hits_at_1000_filtered': 1.0

In [10]:
eval.all_ranks

{'sp_to_o': tensor([ 3,  0, 12, 28,  0,  1,  0]),
 'po_to_s': tensor([ 6, 90,  8,  2, 68, 87,  0]),
 'so_to_p': tensor([ 4,  1,  5,  6, 28, 16,  6])}

In [11]:
eval.all_ranks_filt

{'sp_to_o': tensor([0, 0, 0, 0, 0, 0, 0]),
 'po_to_s': tensor([0, 0, 0, 0, 0, 0, 0]),
 'so_to_p': tensor([ 4,  1,  5,  6, 27, 14,  5])}

In [15]:
valid_data[[17, 18, 19, 20, 136, 20451, 12366]]

tensor([[  34,    6,   35],
        [  36,   16,   32],
        [  37,    8,   38],
        [  39,    7,   40],
        [ 248,   66,  249],
        [1798,   66, 1327],
        [1798,   45, 4657]], dtype=torch.int32)

In [14]:
print(model.dataset.entity_strings(torch.Tensor([1798]).long()))
print(model.dataset.relation_strings(torch.Tensor([66]).long()))
print(model.dataset.entity_strings(torch.Tensor([1327]).long()))

['Zooey Deschanel']
['/people/person/places_lived./people/place_lived/location']
['Los Angeles']


In [14]:
list(predicted_o_score)

[[15.832500457763672,
  15.68239974975586,
  15.575333595275879,
  15.557999610900879,
  15.478597640991211,
  15.47299861907959,
  15.407565116882324,
  15.392574310302734,
  15.387632369995117,
  15.363303184509277,
  15.304561614990234,
  15.169096946716309,
  12.692606925964355,
  12.482621192932129,
  12.355673789978027]]